In [5]:
import openai
import pandas as pd
import numpy as np
import re

import sqlite3

In [46]:
with open('gpt_key.txt', 'r') as file:
    openai.api_key = file.read().rstrip()

# store previous responses in sqlite database to avoid re-prompting
# try to load from database, otherwise create new
sql_conn = sqlite3.connect('carbon_impacts.db')
sql_conn.execute('CREATE TABLE IF NOT EXISTS carbon_impacts (meal TEXT, impact INTEGER, explanation TEXT)')
sql_conn.commit()

In [51]:
def add_response(meal, impact, response, verbose=False):
    # add response to dataframe using concat
    # self.response_df = pd.concat([self.response_df, pd.DataFrame({'meal': [meal], 'response': [response]})])

    # add response to sqlite database
    try:
        sql_conn.execute(f'INSERT INTO carbon_impacts VALUES ("{meal}", {impact}, "{response}")')
        sql_conn.commit()
    except:
        if verbose:
            print('Error adding value to database')

def retrieve_impact(meal, verbose=False):
    # retrieve impact if already run
    cursor = sql_conn.execute(f'SELECT impact FROM carbon_impacts WHERE meal="{meal}"')
    response = cursor.fetchone()
    if response is not None:
        if verbose:
            print('Retrieved response from database')
        return response[0]
    else:
        return None

def get_prompt(prompt_type):
    prompt_types = {
        'Carbon_impact': 'Please rate the following proposed meal on a scale of -10 to 10, where\
                -10 is a meal with a very high carbon impact and 10 is a meal with a very low carbon impact,\
                and 0 is a meal with a neutral carbon impact. Please phrase your numeric answer as: "The carbon impact score is X"\n',
    }  
    return prompt_types[prompt_type]

def ing_to_string(ingredients):
    ing_string = ''
    for k, _ in ingredients.items():
        ing_string += k.name + ','
    return ing_string[:-1]

def craft_prompt(meal, prompt_type='Carbon_impact', verbose=False):
    prompt = get_prompt(prompt_type)
    prompt += f'Meal: {meal.name}\n'
    prompt += f'Ingredients: {ing_to_string(meal.ingredients)}\n'

    if verbose:
        print(prompt)
    return prompt

def parse_carbon_impact(response):
    # try to find a number in the response
    response = response.lower()
    try:
        # 'the carbon impact score is -X'
        num = re.search(r'-\d+',re.search(r'the carbon impact score is -\d+', response).group()).group()
    except:
        try:
            # 'the carbon impact score is X'
            num = re.search(r'\d+',re.search(r'the carbon impact score is \d+', response).group()).group()
        except:
            try:
                # 'a -X'
                num = re.search(r'-\d+',response).group()
            except:
                # 'a X'
                try:
                    num = re.search(r'\d+', response).group()
                except:
                    num = None
                
    return int(num) if num is not None else None

def meal_impact_from_chatgpt(chosen_meal, verbose=False, person_type='Carbon_impact', save_response=True, samples=3):
    """
    Given a proposed meal, get impact from the chatgpt model.
    """
    # print(chosen_meal_string)
    prompt = craft_prompt(chosen_meal, person_type, verbose=verbose)

    # check if we've checked this meal before,
    # if so, retrieve the response (no need to requery)
    avg_impact = retrieve_impact(chosen_meal)
    if avg_impact is not None:
        if verbose:
            print(avg_impact)
        return avg_impact
    else:
        if verbose:
            print('No response found, querying chatgpt...')
        avg_impact = 0
        all_responses = ""
        for i in range(samples):
            resp = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                        {"role": "system", "content": "You are a helpful assistant and an expert in food and in its carbon impact on the environment."},
                        {"role": "user", "content": prompt},
                    ]
            )
            response = resp['choices'][0]['message']['content']
            numeric_impact = parse_carbon_impact(response)

            if verbose:
                print(response)

            if numeric_impact is None:
                print("No numeric, continue")
                continue
            
            avg_impact += numeric_impact
            all_responses += response + '\n'
        
        avg_impact /= samples
        if save_response:
            add_response(chosen_meal, avg_impact, response)
        
    return avg_impact

In [7]:
from dietkit import load_ingredient, load_menu, load_diet

all_ingredients = load_ingredient(sample_language = 'eng')
meal_dict = load_menu(ingredients = all_ingredients, sample_language = 'eng')

possible_meals = np.array(list(meal_dict.keys()))



In [17]:
ing_to_string(meal_dict[possible_meals[0]].ingredients)


'Carbonated beverages, Sprite,Watermelon, Red pulp, Raw'

In [38]:
prompt = craft_prompt(meal_dict[possible_meals[1]])
print(parse_carbon_impact(prompt))

resp = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                    {"role": "system", "content": "You are a helpful assistant and an expert in food and in its carbon impact on the environment."},
                    {"role": "user", "content": prompt},
                ]
        )
response = resp['choices'][0]['message']['content']
print(response)

-10
The carbon impact score is 8. Dried persimmon has a very low carbon impact as it requires minimal processing and transportation compared to other foods. Additionally, persimmons are a seasonal fruit and can be dried to preserve them for longer periods of time, reducing food waste.


In [40]:
print(prompt)

Please rate the following proposed meal on a scale of -10 to 10, where                -10 is a meal with a very high carbon impact and 10 is a meal with a very low carbon impact,                and 0 is a meal with a neutral carbon impact. Please phrase your numeric answer as: "The carbon impact score is X"
Meal: Dried persimmon
Ingredients: Persimmon(Diospyros kaki Thunb.), Dried, Cubed



In [53]:
for meal in possible_meals:
    meal_impact_from_chatgpt(meal_dict[meal], verbose=True)

Please rate the following proposed meal on a scale of -10 to 10, where                -10 is a meal with a very high carbon impact and 10 is a meal with a very low carbon impact,                and 0 is a meal with a neutral carbon impact. Please phrase your numeric answer as: "The carbon impact score is X"
Meal: S watermelon punch (without milks)
Ingredients: Carbonated beverages, Sprite,Watermelon, Red pulp, Raw

No response found, querying chatgpt...
The carbon impact score is 2. While watermelon has a relatively low carbon footprint, carbonated beverages like Sprite have a high carbon footprint due to their production and transportation. However, since the recipe does not include milk, which has a significant carbon footprint, the overall score is not as low as it could be.
The carbon impact score is 2. While watermelon is a low carbon impact fruit, the carbonated beverages and Sprite have a high carbon impact due to their production and transportation process.
The carbon impact sc

AttributeError: 'NoneType' object has no attribute 'group'

In [54]:
#convert carbon_impacts.db to dataframe
carbon_impacts_df = pd.read_sql_query("SELECT * FROM carbon_impacts", sql_conn)

carbon_impacts_df


,meal,impact,explanation
0,Menu object: Dried persimmon,8.000000,The carbon impact score is 9. Drying persimmon...
1,Menu object: S watermelon punch (without milks),-0.333333,The carbon impact score is -5. \n\nCarbonated ...
2,Menu object: S kyogo grape (100g),4.000000,The carbon impact score is 6. Grapes have a re...
3,Menu object: S dried persimmon,7.000000,The carbon impact score is 7. Dried persimmon ...
4,Menu object: S fruit punch,-2.666667,The carbon impact score is -5. \n\nThis meal c...
5,Menu object: S tangerine (50g),9.333333,The carbon impact score is 9. The cultivation ...
6,Menu object: S mandarin (80g),9.000000,The carbon impact score is 9. Mandarins have a...
7,Menu object: S sweet persimmon (50g),6.666667,The carbon impact score is 3. Persimmons have ...
8,Menu object: S carrot stick,9.000000,The carbon impact score is 10. \n\nCarrots hav...
9,Menu object: S strawberry (100g),8.333333,The carbon impact score for this meal is 9. St...
